In [1]:
#utils
import torch
import torch.nn as nn
import numpy as np

In [ ]:
# Convolution(3x3) layer
#in channel, out channel, kernel size, stride값, padding값, bias=편향off
#편향값은 계산하지 않음->메모리 사용량을 줄이기 위함

def conv3x3(in_planes,out_planes,stride=1):
    return nn.Conv2d(in_planes,out_planes,kernel_size=3,stride=stride,padding=1,bias=False) 

In [ ]:
class BasicBlock(nn.Module):
    expansion=1
    
    def __init__(self,inplanes,planes,stride=1,downsample=None):
        super(BasicBlock,self).__init__()
        self.conv1=conv3x3(inplanes,planes,stride) #3x3 convolution
        self.bn1=nn.BatchNorm2d(planes) #batch normalization
        self.relu=nn.ReLU(inplace=True)
        self.conv2=conv3x3(planes,planes)#conv1의 channel값 input, output으로 입력
        self.bn2=nn.BatchNorm2d(planes)
        self.downsample=downsample
        self.stride=stride
    
    def forward(self,x):
        residual=x #x는 input 이미지
        
        out=self.relu(self.bn1(self.conv1(x)))
        out=self.bn2(self.conv2(out))
        
        if self.downsample is not None:
            residual=self.downsample(x)
         
        out+=residual #이거 더해주는 이유 알아보기
        out=self.relu(out)
        return out 

In [ ]:
class Bottleneck(nn.Module):
    expansion=4
    
    def __init__(self,inplanes,planes,stride=1,downsample=None):
        super(Bottleneck,self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * Bottleneck.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * Bottleneck.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        
    def forward(self,x):
        residual=x
        out=self.relu(self.bn1(self.conv1(x)))
        out=self.relu(self.bn2(self.conv2(out)))
        out=self.bn3(self.conv3(out))
        
        if self.downsample is not None:
            residual=self.downsample(x)
            
        out+=residual #이거 더해주는 이유 알아보기
        out=self.relu(out)
        
        return out

In [ ]:
# Bbox 생성
class BBoxTransform(nn.Module):
    def __init__(self,mean=None, std=None):
        super(BBoxTransform,self).__init__()
        if mean is None: #mean이 없을 때
            if torch.cuda.is_available(): #cuda 사용시
                self.mean=torch.from_numpy(np.array([0,0,0,0]).astype(np.float32)).cuda() #0이 4개인 tensor형 자료 만들어줌
            else:
                self.mean=torch.from_numpy(np.array([0,0,0,0])).astype(np.float32)
                
        else:
            self.mean=mean
        if std is None:
            if torch.cuda.is_available():
                self.std=torch.from_numpy(np.array([0.1,0.1,0.2,0.2]).astype(np.float32)).cuda()
            else:
                self.std=torch.from_numpy(np.array([0.1,0.1,0.2,0.2]).astype(np.float32))
        else:
            self.std=std
            
    def forward(self,boxes,deltas): #deltas는 무엇인지?
        widths=boxes[:,:,2]-boxes[:,:,0] #XBR-XTL
        heights=boxes[:,:,3]-boxes[:,:,1] #YTL-YBR
        ctr_x=boxes[:,:,0]+0.5*widths #x의 중앙값
        ctr_y=boxes[:,:,1]+0.5*heights #y중앙값
        
        dx=deltas[:,:,0] * self.std[0]+self.mean[0]
        dy=deltas[:,:,1] * self.std[1]+self.mean[1]
        dw=deltas[:,:,2] * self.std[2]+self.mean[2]
        dh=deltas[:,:,3] * self.std[3]+self.mean[3]
        
        pred_ctr_x=ctr_x+dx*widths
        pred_ctr_y=ctr_y+dy*heights
        pred_w=torch.exp(dw)*widths
        pred_h=torch.exp(dh)*heights
        
        pred_boxes_x1=pred_ctr_x-0.5*pred_w
        pred_boxes_y1=pred_ctr_y-0.5*pred_h
        pred_boxes_x2=pred_ctr_x+0.5*pred_w
        pred_boxes_y2=pred_ctr_y+0.5*pred_h
        
        pred_boxes=torch.stack([pred_boxes_x1,pred_boxes_y1,pred_boxes_x2,pred_boxes_y2],dim=2)
        return pred_boxes

In [ ]:
class ClipBoxes(nn.Module):
    def __init__(self,width=None,height=None):
        super(ClipBoxes,self).__init__()
        
    def forward(self,boxes,img):
        batch_size,num_channels,height,width=img.shape
        
        boxes[:,:,0]=torch.clamp(boxes[:,:,0],min=0) #xtl값이 0보다 작은경우 0으로 반환해줌
        boxes[:,:,1]=torch.clamp(boxes[:,:,1],min=0) #ytl값이 0보다 작은경우 0으로 반환햐줌
        
        boxes[:, :, 2] = torch.clamp(boxes[:, :, 2], max=width) #width값 보다 크면 width로 대체
        boxes[:, :, 3] = torch.clamp(boxes[:, :, 3], max=height) #height값 보다 크면 height로 대체
        
        return boxes